# Industrial Digital Twin by Transformer - Training & Inference

This notebook demonstrates how to train and use the V1 and V4 Transformer models for industrial sensor prediction.

## Features
- **V1 Model**: Static sensor-to-sensor mapping using Transformer architecture
- **V4 Model**: Hybrid temporal + static approach for time-dependent sensor signals

## Quick Start
1. Upload your CSV dataset to `data/raw/` folder
2. Run cells sequentially to train models
3. Evaluate and visualize results

## 1. Setup and Imports

In [ ]:
# Install required packages if running in Colab
import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    !git clone https://github.com/YOUR_USERNAME/Industrial-digital-twin-by-transformer.git
    %cd Industrial-digital-twin-by-transformer
    !pip install -r requirements.txt

In [ ]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# Import models and utilities
from models.v1_transformer import CompactSensorTransformer
from models.v4_hybrid_transformer import HybridTemporalTransformer
from models.utils import create_temporal_context_data, apply_ifd_smoothing
from src.data_loader import SensorDataLoader
from src.trainer import ModelTrainer
from src.inference import ModelInference

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

## 2. Load Dataset

Upload your CSV file to `data/raw/` folder or specify the path below.

In [ ]:
# Option 1: Load from file
data_path = 'data/raw/your_sensor_data.csv'  # Update this path

# Option 2: Use pre-loaded DataFrame (if you have one)
# df = pd.read_csv('your_data.csv')
# data_loader = SensorDataLoader(df=df)

data_loader = SensorDataLoader(data_path=data_path)

# Display data info
print("Dataset Info:")
print(data_loader.get_data_info())

# Get available signals
available_signals = data_loader.get_available_signals()
print(f"\nAvailable signals ({len(available_signals)}):")
for i, signal in enumerate(available_signals[:10], 1):
    print(f"  {i}. {signal}")
if len(available_signals) > 10:
    print(f"  ... and {len(available_signals) - 10} more")

## 3. Configure Model and Signals

Select which sensors to use as inputs (boundary conditions) and outputs (targets).

In [ ]:
# Select signals - UPDATE THESE BASED ON YOUR DATA
boundary_signals = available_signals[:5]  # First 5 signals as inputs
target_signals = available_signals[5:10]   # Next 5 signals as outputs

print("Boundary signals (inputs):")
for sig in boundary_signals:
    print(f"  - {sig}")

print("\nTarget signals (outputs):")
for sig in target_signals:
    print(f"  - {sig}")

In [ ]:
# Prepare data
data_splits = data_loader.prepare_data(
    boundary_signals=boundary_signals,
    target_signals=target_signals,
    test_size=0.2,
    val_size=0.2,
    random_state=42
)

print("Data split:")
print(f"  Training samples:   {len(data_splits['X_train'])}")
print(f"  Validation samples: {len(data_splits['X_val'])}")
print(f"  Test samples:       {len(data_splits['X_test'])}")

## 4. Train V1 Model (Static Transformer)

V1 model learns static relationships between sensors without temporal dependencies.

In [ ]:
# V1 Model Configuration
v1_config = {
    'd_model': 128,
    'nhead': 8,
    'num_layers': 3,
    'dropout': 0.1,
    'lr': 0.001,
    'weight_decay': 1e-5,
    'epochs': 100,
    'batch_size': 64,
    'grad_clip': 1.0,
    'early_stop_patience': 25,
    'scheduler_patience': 10,
    'scheduler_factor': 0.5
}

# Create V1 model
v1_model = CompactSensorTransformer(
    num_boundary_sensors=len(boundary_signals),
    num_target_sensors=len(target_signals),
    d_model=v1_config['d_model'],
    nhead=v1_config['nhead'],
    num_layers=v1_config['num_layers'],
    dropout=v1_config['dropout']
)

print(f"V1 Model created with {sum(p.numel() for p in v1_model.parameters()):,} parameters")

In [ ]:
# Create data loaders
train_loader, val_loader = data_loader.create_dataloaders(
    data_splits['X_train'],
    data_splits['y_train'],
    data_splits['X_val'],
    data_splits['y_val'],
    batch_size=v1_config['batch_size']
)

# Train V1 model
v1_trainer = ModelTrainer(v1_model, device=str(device), config=v1_config)
v1_history = v1_trainer.train(train_loader, val_loader, verbose=True)

In [ ]:
# Plot training history
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(v1_history['train_losses'], label='Training Loss')
plt.plot(v1_history['val_losses'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('V1 Model Training History')
plt.legend()
plt.grid(True)

plt.subplot(1, 2, 2)
plt.plot(np.log10(v1_history['train_losses']), label='Log Training Loss')
plt.plot(np.log10(v1_history['val_losses']), label='Log Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Log10(Loss)')
plt.title('V1 Model Training History (Log Scale)')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

## 5. Evaluate V1 Model

In [ ]:
# Create inference engine
v1_inference = ModelInference(
    model=v1_model,
    scaler_X=data_splits['scaler_X'],
    scaler_y=data_splits['scaler_y'],
    device=str(device)
)

# Evaluate on test set
X_test_original = data_splits['scaler_X'].inverse_transform(data_splits['X_test'])
y_test_original = data_splits['scaler_y'].inverse_transform(data_splits['y_test'])

v1_metrics = v1_inference.evaluate(X_test_original, y_test_original, target_signals)
v1_inference.print_metrics(v1_metrics)

In [ ]:
# Visualize predictions for first 3 target signals
fig = v1_inference.plot_predictions(
    X_test_original[:1000],
    y_test_original[:1000],
    signal_indices=[0, 1, 2],
    target_signal_names=target_signals
)
plt.show()

## 6. Train V4 Model (Hybrid Temporal + Static)

V4 model combines temporal context analysis with static mapping for improved accuracy.

In [ ]:
# V4 Model Configuration
v4_config = {
    'd_model': 64,
    'nhead': 4,
    'num_layers': 2,
    'dropout': 0.1,
    'context_window': 5,
    'use_temporal': True,
    'lr': 0.001,
    'weight_decay': 1e-5,
    'epochs': 100,
    'batch_size': 64,
    'grad_clip': 1.0,
    'early_stop_patience': 25,
    'scheduler_patience': 10,
    'scheduler_factor': 0.5
}

# Create temporal context data
X_train_ctx, y_train_ctx, _ = create_temporal_context_data(
    data_splits['X_train'],
    data_splits['y_train'],
    context_window=v4_config['context_window']
)

X_val_ctx, y_val_ctx, _ = create_temporal_context_data(
    data_splits['X_val'],
    data_splits['y_val'],
    context_window=v4_config['context_window']
)

print(f"Temporal context data shapes:")
print(f"  Training: {X_train_ctx.shape}")
print(f"  Validation: {X_val_ctx.shape}")

In [ ]:
# Create V4 model
v4_model = HybridTemporalTransformer(
    num_boundary_sensors=len(boundary_signals),
    num_target_sensors=len(target_signals),
    d_model=v4_config['d_model'],
    nhead=v4_config['nhead'],
    num_layers=v4_config['num_layers'],
    dropout=v4_config['dropout'],
    use_temporal=v4_config['use_temporal'],
    context_window=v4_config['context_window']
)

print(f"V4 Model created with {sum(p.numel() for p in v4_model.parameters()):,} parameters")

In [ ]:
# Create data loaders for V4
train_loader_v4, val_loader_v4 = data_loader.create_dataloaders(
    X_train_ctx,
    y_train_ctx,
    X_val_ctx,
    y_val_ctx,
    batch_size=v4_config['batch_size']
)

# Train V4 model
v4_trainer = ModelTrainer(v4_model, device=str(device), config=v4_config)
v4_history = v4_trainer.train(train_loader_v4, val_loader_v4, verbose=True)

In [ ]:
# Plot V4 training history
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(v4_history['train_losses'], label='Training Loss')
plt.plot(v4_history['val_losses'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('V4 Model Training History')
plt.legend()
plt.grid(True)

plt.subplot(1, 2, 2)
plt.plot(np.log10(v4_history['train_losses']), label='Log Training Loss')
plt.plot(np.log10(v4_history['val_losses']), label='Log Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Log10(Loss)')
plt.title('V4 Model Training History (Log Scale)')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

## 7. Compare Models

Compare V1 and V4 model performance.

In [ ]:
# Prepare V4 test data with temporal context
X_test_ctx, y_test_ctx, _ = create_temporal_context_data(
    data_splits['X_test'],
    data_splits['y_test'],
    context_window=v4_config['context_window']
)

# Evaluate V4 model
v4_model.eval()
with torch.no_grad():
    X_test_tensor = torch.FloatTensor(X_test_ctx).to(device)
    y_pred_v4_scaled = v4_model(X_test_tensor).cpu().numpy()

y_pred_v4 = data_splits['scaler_y'].inverse_transform(y_pred_v4_scaled)
y_test_v4 = data_splits['scaler_y'].inverse_transform(y_test_ctx)

# Compute metrics for V4
from sklearn.metrics import r2_score, mean_squared_error

print("\n" + "="*80)
print("MODEL COMPARISON")
print("="*80)

for i, signal in enumerate(target_signals[:5]):  # Compare first 5 signals
    v1_r2 = v1_metrics[signal]['R2']
    v1_rmse = v1_metrics[signal]['RMSE']
    
    v4_r2 = r2_score(y_test_v4[:, i], y_pred_v4[:, i])
    v4_rmse = np.sqrt(mean_squared_error(y_test_v4[:, i], y_pred_v4[:, i]))
    
    print(f"\n{signal[:50]}:")
    print(f"  V1 - R²: {v1_r2:.4f}, RMSE: {v1_rmse:.4f}")
    print(f"  V4 - R²: {v4_r2:.4f}, RMSE: {v4_rmse:.4f}")
    print(f"  Improvement: R² {(v4_r2-v1_r2)*100:+.2f}%, RMSE {(v4_rmse-v1_rmse)/v1_rmse*100:+.2f}%")

print("\n" + "="*80)

## 8. Save Models

Save trained models for later use.

In [ ]:
# Save V1 model
v1_trainer.save_model('models/saved/v1_model.pth')
print("✅ V1 model saved to models/saved/v1_model.pth")

# Save V4 model
v4_trainer.save_model('models/saved/v4_model.pth')
print("✅ V4 model saved to models/saved/v4_model.pth")

## 9. Load Saved Models (Optional)

Load previously saved models for inference.

In [ ]:
# Load V1 model
# v1_trainer_loaded = ModelTrainer(v1_model, device=str(device))
# v1_trainer_loaded.load_model('models/saved/v1_model.pth')
# print("✅ V1 model loaded")

# Load V4 model
# v4_trainer_loaded = ModelTrainer(v4_model, device=str(device))
# v4_trainer_loaded.load_model('models/saved/v4_model.pth')
# print("✅ V4 model loaded")